In [1]:

import os
import numpy as np
import torch
from torch import nn
from scipy.stats import ortho_group
from typing import Union
from typing_extensions import Literal
from pytorch_lightning import LightningModule, Trainer
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchmetrics import Accuracy
from torchvision import transforms

from torchvision.datasets import MNIST
PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 10000 if AVAIL_GPUS else 64
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
from scipy.linalg import block_diag
from sklearn.linear_model import LinearRegression
from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('font',**{'family':'serif','serif':['Times']})
rc('text', usetex=True)
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

## Blockwise MCC (instead of summing |correlations| this sums R2 across blocks)

In [2]:
def MCC_block_r2(Z,Zp,b):
    n = np.shape(Z)[1]
    s = np.arange(0,n,b)
    l = np.shape(s)[0]
    R2_matrix = np.zeros((l,l))
    c=0
    for i in list(s):
        d=0
        for j in list(s):
            
            reg = LinearRegression().fit(Z[:,i:i+b], Zp[:,j:j+b])
            R2_matrix[c,d] = reg.score(Z[:,i:i+b], Zp[:,j:j+b])
            d=d+1
        c=c+1
    print(R2_matrix)
    
    r,c = linear_sum_assignment(-R2_matrix)
    
    return np.mean(R2_matrix[r,c])

In [3]:
def MCC(Z,Zp):
    n = np.shape(Z)[1]
#     print (n)
    rho_matrix = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            rho_matrix[i,j] = np.abs(np.corrcoef(Z[:,i], Zp[:,j])[0,1])
    
    r,c = linear_sum_assignment(-rho_matrix)
    
    return np.mean(rho_matrix[r,c])

## Invertible MLP function from CIDGP paper

In [4]:
## Invertible MLP function from CIDGP paper

def construct_invertible_mlp(
    n: int = 20,
    n_layers: int = 2,
    n_iter_cond_thresh: int = 10000,
    cond_thresh_ratio: float = 0.25,
    weight_matrix_init: Union[Literal["pcl"], Literal["rvs"]] = "pcl",
    act_fct: Union[
        Literal["relu"],
        Literal["leaky_relu"],
        Literal["elu"],
        Literal["smooth_leaky_relu"],
        Literal["softplus"],
    ] = "leaky_relu",
):
   
    """
    Create an (approximately) invertible mixing network based on an MLP.
    Based on the mixing code by Hyvarinen et al.
    Args:
        n: Dimensionality of the input and output data
        n_layers: Number of layers in the MLP.
        n_iter_cond_thresh: How many random matrices to use as a pool to find weights.
        cond_thresh_ratio: Relative threshold how much the invertibility
            (based on the condition number) can be violated in each layer.
        weight_matrix_init: How to initialize the weight matrices.
        act_fct: Activation function for hidden layers.
    """

    class SmoothLeakyReLU(nn.Module):
        def __init__(self, alpha=0.2):
            super().__init__()
            self.alpha = alpha

        def forward(self, x):
            return self.alpha * x + (1 - self.alpha) * torch.log(1 + torch.exp(x))

    def get_act_fct(act_fct):
        if act_fct == "relu":
            return torch.nn.ReLU, {}, 1
        if act_fct == "leaky_relu":
            return torch.nn.LeakyReLU, {"negative_slope": 0.2}, 1
        elif act_fct == "elu":
            return torch.nn.ELU, {"alpha": 1.0}, 1
        elif act_fct == "max_out":
            raise NotImplemented()
        elif act_fct == "smooth_leaky_relu":
            return SmoothLeakyReLU, {"alpha": 0.2}, 1
        elif act_fct == "softplus":
            return torch.nn.Softplus, {"beta": 1}, 1
        else:
            raise Exception(f"activation function {act_fct} not defined.")

    layers = []
    act_fct, act_kwargs, act_fac = get_act_fct(act_fct)

    # Subfuction to normalize mixing matrix
    def l2_normalize(Amat, axis=0):
        # axis: 0=column-normalization, 1=row-normalization
        l2norm = np.sqrt(np.sum(Amat * Amat, axis))
        Amat = Amat / l2norm
        return Amat
    print (n_iter_cond_thresh)
    condList = np.zeros([n_iter_cond_thresh])
    if weight_matrix_init == "pcl":
        for i in range(n_iter_cond_thresh):
            A = np.random.uniform(-1, 1, [n, n])
            A = l2_normalize(A, axis=0)
            condList[i] = np.linalg.cond(A)
        condList.sort()  # Ascending order
    condThresh = condList[int(n_iter_cond_thresh * cond_thresh_ratio)]
    print("condition number threshold: {0:f}".format(condThresh))

    for i in range(n_layers):

        lin_layer = nn.Linear(n, n, bias=False)

        if weight_matrix_init == "pcl":
            condA = condThresh + 1
            while condA > condThresh:
                weight_matrix = np.random.uniform(-1, 1, (n, n))
                weight_matrix = l2_normalize(weight_matrix, axis=0)

                condA = np.linalg.cond(weight_matrix)
                # print("    L{0:d}: cond={1:f}".format(i, condA))
            print(
                f"layer {i+1}/{n_layers},  condition number: {np.linalg.cond(weight_matrix)}"
            )
            lin_layer.weight.data = torch.tensor(weight_matrix, dtype=torch.float32)

        elif weight_matrix_init == "rvs":
            weight_matrix = ortho_group.rvs(n)
            lin_layer.weight.data = torch.tensor(weight_matrix, dtype=torch.float32)
        elif weight_matrix_init == "expand":
            pass
        else:
            raise Exception(f"weight matrix {weight_matrix_init} not implemented")

        layers.append(lin_layer)

        if i < n_layers - 1:
            layers.append(act_fct(**act_kwargs))

    mixing_net = nn.Sequential(*layers)

    # fix parameters
    for p in mixing_net.parameters():
        p.requires_grad = False

    return mixing_net

## Data generation function 

In [5]:
def data_generation_MLP(n, n_layers, cond_thresh_ratio, n_iter_cond_thresh, n_data, distribution):
    
    if(distribution == "uniform_non_linear_block"): # all components of z are uniform i.i.d.
        data_Z        = np.random.rand(n_data, n)*5.0
        tensor_Z      = torch.Tensor(data_Z)


    if(distribution == "normal_non_linear_block"): # all components of z are uniform i.i.d.
        A1 = np.ones((n//2,n//2))
        A2 = np.ones((n//2,n//2))
        A = block_diag(A1,A2)
        data_Z = np.random.multivariate_normal(mean=np.zeros(n), cov=np.eye(n)+ A@A.T, size=n_data, check_valid='warn', tol=1e-8)
        tensor_Z      = torch.Tensor(data_Z)

    g = construct_invertible_mlp(n, n_layers,   n_iter_cond_thresh, cond_thresh_ratio)  
    data = g(tensor_Z)


    data_Z_list   = [tensor_Z]
    data_X_list     = [data]
    offset_1 = torch.ones(n_data,2)
    offset_2 = torch.cat((torch.ones((n_data,1)), 2.0*torch.ones((n_data,1))), axis=1)

    for i in range(n):

        if(i<=n//2-1):

            tensor_Z_copy = torch.clone(tensor_Z)
            tensor_Z_copy[:,2*i:2*i+2] = tensor_Z_copy[:,2*i:2*i+2] + offset_1
            data_copy = g(tensor_Z_copy)
            data_Z_list.append(tensor_Z_copy)
            data_X_list.append(data_copy)
        else:
            j = i-n//2
            tensor_Z_copy = torch.clone(tensor_Z)
            tensor_Z_copy[:,2*j:2*j+2] = tensor_Z_copy[:,2*j:2*j+2] + offset_2
            data_copy = g(tensor_Z_copy)
            data_Z_list.append(tensor_Z_copy)
            data_X_list.append(data_copy)                   

    return data_X_list, data_Z_list  
        

## Construct the model and loss (compares intervened and default latent)

In [6]:
class Latent_comparison_model(LightningModule):
    def __init__(self,n):
        super().__init__()
        self.n     = n
        num_hidden = 100
        self.linear1 = nn.Linear(self.n, num_hidden)
        self.act_fn1 = nn.ReLU()
        self.linear2 = nn.Linear(num_hidden, num_hidden)
        self.act_fn2 = nn.ReLU()
        self.linear3 = nn.Linear(num_hidden, self.n)
        
    def forward(self, x):
       
        x = self.linear1(x)
        x = self.act_fn1(x)
        x = self.linear2(x)
        x = self.act_fn2(x)
        x = self.linear3(x)
        
        return x

    def training_step(self, batch_X_list, batch_idx):
        L      = len(batch_X_list)
        X_0    = batch_X_list[0]
        n_d    = X_0.size()[0]
        offset = torch.ones(n_d)
        loss   = 0.0      
        offset_matrix1 = np.random.RandomState(1).uniform(1, 5, size=(self.n,2))
        offset_matrix2 = np.random.RandomState(2).uniform(1, 5, size=(self.n,2))
#       
        i= np.random.randint(L-1)
        if(i <=(L-1)//2-1):
            offset_1 = torch.ones((n_d,2))*offset_matrix1[i]
            Z_target  = self(batch_X_list[i+1])
            Z_predict = self(X_0) 
            Z_predict[:,2*i:2*i+2] = Z_predict[:,2*i:2*i+2] + offset_1 
            loss = loss + F.mse_loss(Z_predict, Z_target)
        else:
            offset_2 = torch.ones((n_d,2))*offset_matrix2[i]
            j=i-(L-1)//2
            Z_target  = self(batch_X_list[i+1])
            Z_predict = self(X_0) 
            Z_predict[:,2*j:2*j+2] = Z_predict[:,2*j:2*j+2] + offset_2 
            loss = loss + F.mse_loss(Z_predict, Z_target)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch_X_list, batch_idx):
        L      = len(batch_X_list)
        X_0    = batch_X_list[0]
        n_d    = X_0.size()[0]
        offset = torch.ones(n_d)
        loss   = 0.0      
        offset_matrix1 = np.random.RandomState(1).uniform(1, 5, size=(self.n,2))
        offset_matrix2 = np.random.RandomState(2).uniform(1, 5, size=(self.n,2))
#       
        i= np.random.randint(L-1)
        if(i <=(L-1)//2-1):
            offset_1 = torch.ones((n_d,2))*offset_matrix1[i]
            Z_target  = self(batch_X_list[i+1])
            Z_predict = self(X_0) 
            Z_predict[:,2*i:2*i+2] = Z_predict[:,2*i:2*i+2] + offset_1 
            loss = loss + F.mse_loss(Z_predict, Z_target)
        else:
            offset_2 = torch.ones((n_d,2))*offset_matrix2[i]
            j=i-(L-1)//2
            Z_target  = self(batch_X_list[i+1])
            Z_predict = self(X_0) 
            Z_predict[:,2*j:2*j+2] = Z_predict[:,2*j:2*j+2] + offset_2 
            loss = loss + F.mse_loss(Z_predict, Z_target)
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.005)

In [20]:
np.random.randint(5)

4

## Generate data

In [ ]:
n=10
data_X, data_Z = data_generation_MLP(n=n, n_layers=1, cond_thresh_ratio=0.0, n_iter_cond_thresh=25000, n_data=25000, distribution="uniform_non_linear_block")

k= len(data_X)
train_data_X = []
test_data_X  = []
train_data_Z = []
test_data_Z  = []
train_size = 20000
for i in range(k):
    train_data_X.append(data_X[i][:train_size])
    train_data_Z.append(data_Z[i][:train_size])
    test_data_X.append(data_X[i][train_size:])
    test_data_Z.append(data_Z[i][train_size:])

## Train model 

In [46]:
latent_comparison_model = Latent_comparison_model(n)
train_loader = DataLoader(train_data_X, batch_size=BATCH_SIZE)
val_loader = DataLoader(test_data_X, batch_size=BATCH_SIZE)
trainer = Trainer(
    gpus=AVAIL_GPUS,
    max_epochs=2000,
    progress_bar_refresh_rate=50)
# trainer = Trainer(
#     gpus=AVAIL_GPUS,
#     max_epochs=2000,
#     progress_bar_refresh_rate=20, 
# callbacks=[EarlyStopping(monitor="train_loss",   patience=2000)])
trainer.fit(latent_comparison_model, train_loader, val_loader)

/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=50)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name    | Type   | Params
-----------------------------------
0 | linear1 | Linear | 2.1 K 
1 | act_fn1 | ReLU   | 0     
2 | linear2 | Linear | 10.1 K
3 | act_fn2 | ReLU   | 0     
4 | linear3 | Linear | 2.0 K 
-----------------------------------
14.2 K    Trainable params
0         No

Validation sanity check: 0it [00:00, ?it/s]

/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## Output BMCC and R2 matrix 

In [47]:
Zp=latent_comparison_model(test_data_X[0])
Z = test_data_Z[0]
Zp_np= Zp.detach().numpy()
Z_np    = Z.detach().numpy()
MCC_block_r2(Zp_np,Z_np,2)

[[7.46480016e-01 6.57879183e-02 1.48909728e-01 6.79020252e-03
  1.18456366e-02 4.54890217e-03 9.92751815e-03 2.28830571e-02
  5.83515606e-02 7.74566125e-02]
 [3.61295433e-02 6.14671687e-01 2.83507853e-02 3.29370984e-02
  3.32714006e-02 1.28011202e-02 5.97272581e-02 9.43778705e-03
  7.12069360e-03 1.29638870e-02]
 [2.83207768e-02 1.01171732e-02 6.23773752e-01 3.39538093e-03
  4.10784253e-04 1.85347319e-03 3.20089519e-03 5.86395034e-03
  8.24447250e-03 5.49512652e-02]
 [1.62569046e-02 5.03178101e-02 5.39428864e-03 7.90327054e-01
  4.24103125e-03 1.97895121e-02 3.06928334e-02 8.04962855e-03
  1.98540259e-02 3.95252451e-02]
 [1.26173569e-02 3.69243757e-02 1.23374525e-02 3.95635322e-03
  8.92307360e-01 3.09709491e-03 7.46329690e-03 5.09459484e-02
  8.03911902e-03 9.40003752e-03]
 [3.42082359e-02 1.24742517e-01 2.45884657e-02 1.13202933e-01
  2.40469941e-03 8.43676828e-01 2.54273043e-02 1.12484058e-02
  1.79923711e-02 6.57980592e-02]
 [4.34065621e-03 3.71699087e-02 9.11524215e-03 3.86295626e

0.7414328182161449

## Generate results

In [86]:
n_trials=5
dim_list = [6,10,20]
s = len(dim_list)
MCC_result = np.zeros((s,n_trials))
MCC_result1 = np.zeros((s,n_trials))
m=0
for d in dim_list:
    for trial in range(n_trials):
        data_X, data_Z = data_generation_MLP(n=d, n_layers=2, cond_thresh_ratio=0.0, n_iter_cond_thresh=25000, n_data=15000, distribution="normal_non_linear_block")
        k= len(data_X)
        train_data_X = []
        test_data_X  = []
        train_data_Z = []
        test_data_Z  = []
        train_size = 10000
        for i in range(k):
            train_data_X.append(data_X[i][:train_size])
            train_data_Z.append(data_Z[i][:train_size])
            test_data_X.append(data_X[i][train_size:])
            test_data_Z.append(data_Z[i][train_size:])  
        latent_comparison_model = Latent_comparison_model(d)
        train_loader = DataLoader(train_data_X, batch_size=BATCH_SIZE)
        trainer = Trainer(
            gpus=AVAIL_GPUS,
            max_epochs=2000,
            progress_bar_refresh_rate=20,
        )
        trainer.fit(latent_comparison_model, train_loader)
        
        Zp=latent_comparison_model(test_data_X[0])
        Z = test_data_Z[0]
        MCC_result[m][trial] = MCC_block_r2(Z.detach().numpy(),Zp.detach().numpy(),2)
        MCC_result1[m][trial] = MCC(Z.detach().numpy(),Zp.detach().numpy())
    m=m+1
        

25000
condition number threshold: 2.249132
layer 1/2,  condition number: 2.1335541945242467


/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `v

layer 2/2,  condition number: 1.9049187048360066


/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

...

/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `v

layer 2/2,  condition number: 8.517560896126303


/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

[[0.90477893 0.79421274 0.80679346 0.81084223 0.80072256 0.00676847
  0.00522575 0.00634305 0.00382462 0.00721645]
 [0.8275963  0.92246575 0.8400079  0.8308023  0.84153069 0.58306375
  0.62984834 0.57934195 0.56885913 0.58552008]
 [0.83380407 0.84698455 0.91730789 0.83761719 0.85121937 0.6521036
  0.73042646 0.66037362 0.67868487 0.64895678]
 [0.7977408  0.79267694 0.79913342 0.89529172 0.81796656 0.75822353
  0.7888425  0.74458622 0.75950279 0.75468048]
 [0.82378535 0.82223488 0.83389586 0.8341982  0.90841797 0.40621835
  0.4151701  0.40993933 0.38897856 0.42939946]
 [0.34603534 0.4068272  0.38099782 0.39894591 0.39583709 0.90875477
  0.83417944 0.82607741 0.81914014 0.81453093]
 [0.14767883 0.15698604 0.14954183 0.15188708 0.15147753 0.70173718
  0.78577218 0.69767211 0.69709077 0.70302682]
 [0.21531673 0.24961637 0.24870051 0.25619459 0.23334297 0.84818351
  0.84580446 0.92874085 0.84597933 0.8617629 ]
 [0.13261037 0.13047162 0.14279187 0.14466527 0.13518773 0.79555577
  0.80162259 

In [87]:
MCC_result

array([[0.98954384, 0.99504503, 0.99878059, 0.98238797, 0.99219206],
       [0.92904737, 0.87211228, 0.91117383, 0.91149957, 0.89806854],
       [0.89065293, 0.91784071, 0.90129657, 0.891299  , 0.90019336]])

In [21]:
MCC_result = np.array([[0.98954384, 0.99504503, 0.99878059, 0.98238797, 0.99219206],
       [0.92904737, 0.87211228, 0.91117383, 0.91149957, 0.89806854],
       [0.89065293, 0.91784071, 0.90129657, 0.891299  , 0.90019336]])

In [22]:
print ([np.std(MCC_result[i,:]) for i in range(3)])

[0.005529347518597104, 0.01890166038524174, 0.009826042301786827]


In [90]:
MCC_result1

array([[0.89392916, 0.89458213, 0.88975264, 0.90395695, 0.89489194],
       [0.90141572, 0.9119722 , 0.90749416, 0.92033294, 0.89471712],
       [0.93492935, 0.95479402, 0.94409267, 0.93650977, 0.94048117]])

In [26]:
n_trials=5
dim_list = [20]
s = len(dim_list)
MCC_result = np.zeros((s,n_trials))
MCC_result1 = np.zeros((s,n_trials))
m=0
for d in dim_list:
    for trial in range(n_trials):
        data_X, data_Z = data_generation_MLP(n=d, n_layers=2, cond_thresh_ratio=0.0, n_iter_cond_thresh=25000, n_data=15000, distribution="uniform_non_linear_block")
        k= len(data_X)
        train_data_X = []
        test_data_X  = []
        train_data_Z = []
        test_data_Z  = []
        train_size = 10000
        for i in range(k):
            train_data_X.append(data_X[i][:train_size])
            train_data_Z.append(data_Z[i][:train_size])
            test_data_X.append(data_X[i][train_size:])
            test_data_Z.append(data_Z[i][train_size:])  
        latent_comparison_model = Latent_comparison_model(d)
        train_loader = DataLoader(train_data_X, batch_size=BATCH_SIZE)
        trainer = Trainer(
            gpus=AVAIL_GPUS,
            max_epochs=2000,
            progress_bar_refresh_rate=20,
        )
        trainer.fit(latent_comparison_model, train_loader)
        
        Zp=latent_comparison_model(test_data_X[0])
        Z = test_data_Z[0]
        MCC_result[m][trial] = MCC_block_r2(Z.detach().numpy(),Zp.detach().numpy(),2)
        MCC_result1[m][trial] = MCC(Z.detach().numpy(),Zp.detach().numpy())
    
    m=m+1
        

25000
condition number threshold: 8.015397
layer 1/2,  condition number: 7.9580577990915815


/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `v

layer 2/2,  condition number: 8.01259648012401


/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

...

/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=20)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `v

layer 2/2,  condition number: 7.269938624663105


/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

[[0.65446019 0.036049   0.16099856 0.02429194 0.11882384 0.00713523
  0.30904437 0.03526331 0.05058673 0.03449425]
 [0.00411195 0.53360724 0.18867044 0.01769782 0.00436997 0.01725147
  0.10967436 0.00222017 0.00601133 0.02386583]
 [0.00096079 0.01283287 0.37619943 0.01591383 0.03863437 0.05763404
  0.01126253 0.01913274 0.13153921 0.03453413]
 [0.10776291 0.02671951 0.14784231 0.60849016 0.00301201 0.04925167
  0.16567567 0.14377748 0.03344304 0.07987693]
 [0.00390201 0.03866929 0.00291345 0.00617555 0.53320872 0.13507887
  0.01871735 0.01676619 0.0910568  0.04194703]
 [0.00787875 0.16031756 0.02014181 0.00397409 0.01246766 0.56068934
  0.056344   0.08666381 0.10627756 0.00390461]
 [0.06054415 0.08318632 0.0333016  0.23807932 0.0282896  0.05022955
  0.24069898 0.10953838 0.08664137 0.02237813]
 [0.05391428 0.01356918 0.03987493 0.00247626 0.00805782 0.00262428
  0.01617365 0.50250137 0.02230702 0.0319698 ]
 [0.00491373 0.04703411 0.00940006 0.04031707 0.04112838 0.00144756
  0.01797014

In [9]:
MCC_result

array([[0.91994147, 0.98387766, 0.9899733 , 0.90321811, 0.99910882],
       [0.74062168, 0.88985356, 0.80572396, 0.78767303, 0.82787681],
       [0.51750683, 0.50427022, 0.4863096 , 0.        , 0.        ]])

In [27]:
MCC_result

array([[0.59216826, 0.52834015, 0.53390537, 0.47624464, 0.51345755]])

In [30]:
np.mean(MCC_result)

0.5288231918017867

In [31]:
MCC_result = np.array([[0.91994147, 0.98387766, 0.9899733 , 0.90321811, 0.99910882],
       [0.74062168, 0.88985356, 0.80572396, 0.78767303, 0.82787681],
       [0.51750683, 0.50427022, 0.4863096 , 0.        , 0.        ]])

In [32]:
print ([np.std(MCC_result[i,:]) for i in range(2)])

[0.03955729552466669, 0.049042204162843855]


In [29]:
print (np.std(MCC_result))

0.037521371467747575


In [10]:
MCC_result1

array([[0.70649688, 0.71125764, 0.72064383, 0.69955248, 0.73184561],
       [0.6608753 , 0.82065082, 0.7208963 , 0.69183542, 0.7402296 ],
       [0.53641439, 0.51109376, 0.53467918, 0.        , 0.        ]])

In [84]:
MCC_result

array([[0.94856328, 0.86559434, 0.9813423 , 0.94023407, 0.97533127],
       [0.65037268, 0.56176207, 0.63989972, 0.60733358, 0.7760613 ],
       [0.45838021, 0.43282967, 0.44484083, 0.42533508, 0.43285065]])

In [85]:
MCC_result1

array([[0.69821224, 0.7140835 , 0.71758292, 0.70086356, 0.69056526],
       [0.69004596, 0.64436506, 0.68156918, 0.65179109, 0.78528694],
       [0.56605938, 0.55824529, 0.54728067, 0.52500284, 0.54911704]])

In [81]:
n_trials=5
dim_list = [20]
s = len(dim_list)
MCC_result = np.zeros((s,n_trials))
m=0
for d in dim_list:
    for trial in range(n_trials):
        data_X, data_Z = data_generation_MLP(n=d, n_layers=1, cond_thresh_ratio=0.0, n_iter_cond_thresh=25000, n_data=15000, distribution="uniform_non_linear_block")
        k= len(data_X)
        train_data_X = []
        test_data_X  = []
        train_data_Z = []
        test_data_Z  = []
        train_size = 10000
        for i in range(k):
            train_data_X.append(data_X[i][:train_size])
            train_data_Z.append(data_Z[i][:train_size])
            test_data_X.append(data_X[i][train_size:])
            test_data_Z.append(data_Z[i][train_size:])  
        latent_comparison_model = Latent_comparison_model(d)
        train_loader = DataLoader(train_data_X, batch_size=BATCH_SIZE)
        trainer = Trainer(
            gpus=AVAIL_GPUS,
            max_epochs=2000,
            progress_bar_refresh_rate=50,
        )
        trainer.fit(latent_comparison_model, train_loader)
        
        Zp=latent_comparison_model(test_data_X[0])
        Z = test_data_Z[0]
        MCC_result[m][trial] = MCC_block_r2(Z.detach().numpy(),Zp.detach().numpy(),2)
    
    m=m+1

<class 'torch.nn.modules.activation.LeakyReLU'>
25000
condition number threshold: 8.516649


/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=50)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `v

layer 1/1,  condition number: 8.45232926835433


/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

...

/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=50)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `v

layer 1/1,  condition number: 9.602656966735495


/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

[[8.35505483e-01 5.35483296e-02 4.38261971e-02 1.24528716e-02
  5.86266496e-03 2.64898397e-02 5.83223782e-03 3.17020471e-02
  1.13249849e-01 4.65036168e-02]
 [3.25975613e-03 5.99846775e-01 1.31397796e-02 1.18987204e-02
  1.28430532e-02 9.06085696e-04 3.27922796e-03 6.75828153e-02
  9.68043021e-02 9.54050499e-02]
 [3.17506813e-02 2.53444760e-02 7.71809877e-01 1.87429017e-02
  8.42517031e-03 2.32688948e-03 1.92394215e-02 1.96281702e-02
  9.02303375e-03 1.16018289e-01]
 [2.63616409e-03 1.37645709e-02 5.83499943e-03 8.77818267e-01
  6.34158719e-03 2.42783112e-03 7.27680893e-03 3.87434768e-02
  1.12114040e-03 2.96911669e-02]
 [3.94145198e-02 5.53433635e-02 5.42429511e-03 1.03610836e-02
  9.04594927e-01 6.55471049e-02 4.80740982e-02 1.15148244e-01
  4.69009575e-03 2.89192917e-02]
 [1.35656097e-02 3.18512165e-03 3.36569275e-02 2.12433215e-02
  2.44112339e-02 8.49599209e-01 4.57902100e-02 2.11653110e-02
  3.64455716e-02 2.95603486e-03]
 [2.36313526e-02 7.77176161e-02 4.79609739e-02 3.53743649e

In [ ]:
n_trials=5
dim_list = [6,10,20]
s = len(dim_list)
MCC_result = np.zeros((s,n_trials))
MCC_result1 = np.zeros((s,n_trials))
m=0
for d in dim_list:
    for trial in range(n_trials):
        data_X, data_Z = data_generation_MLP(n=d, n_layers=2, cond_thresh_ratio=0.0, n_iter_cond_thresh=25000, n_data=15000, distribution="uniform_non_linear_block")
        k= len(data_X)
        train_data_X = []
        test_data_X  = []
        train_data_Z = []
        test_data_Z  = []
        train_size = 10000
        for i in range(k):
            train_data_X.append(data_X[i][:train_size])
            train_data_Z.append(data_Z[i][:train_size])
            test_data_X.append(data_X[i][train_size:])
            test_data_Z.append(data_Z[i][train_size:])  
        latent_comparison_model = Latent_comparison_model(d)
        train_loader = DataLoader(train_data_X, batch_size=BATCH_SIZE)
        trainer = Trainer(
            gpus=AVAIL_GPUS,
            max_epochs=2000,
            progress_bar_refresh_rate=50,
        )
        trainer.fit(latent_comparison_model, train_loader)
        
        Zp=latent_comparison_model(test_data_X[0])
        Z = test_data_Z[0]
        MCC_result[m][trial] = MCC_block_r2(Z.detach().numpy(),Zp.detach().numpy(),2)
        MCC_result1[m][trial] = MCC(Z.detach().numpy(),Zp.detach().numpy())
    
    m=m+1

<class 'torch.nn.modules.activation.LeakyReLU'>
25000
condition number threshold: 1.847099
layer 1/2,  condition number: 1.820319294061302


/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=50)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `v

layer 2/2,  condition number: 1.7978019966568768


/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

[[0.9681321  0.01186567 0.008028  ]
 [0.00405845 0.95053046 0.01106956]
 [0.03193754 0.03735178 0.98353325]]
<class 'torch.nn.modules.activation.LeakyReLU'>
25000
condition number threshold: 1.874497
layer 1/2,  condition number: 1.8004347520438948


/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=50)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:122: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `v

layer 2/2,  condition number: 1.7595760589153082


/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/Users/kartik/Desktop/causal_latent_DAG/env_DAG/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]